In [ ]:
import re
import pandas as pd
import numpy as np
from datetime import datetime

#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

import requests

import geocoder
import gmplot

In [ ]:
from io import BytesIO
from zipfile import ZipFile
import requests

def get_zip(file_url):
    """Unpack zip file from web and return file objects"""
    url = requests.get(file_url)
    file_objects = ZipFile(BytesIO(url.content))
    return file_objects

# For example:
# zp = get_zip('http://geo-koop.rug.nl/agol/rug/706fd44e86c34b2c9a06ddd477e22b9a/0.zip')
# zp.namelist() to list file names
# If you want to e.g. open a csv file from the zip:
# pd.read_csv(zp.open(file_name))

In [ ]:
# load csv file with all ambulance calls from 2013 - 2017
all_calls_df = pd.read_csv('20170523_ambulancecalls_2013-2017.csv')

In [ ]:
# cleaning
all_calls_df.drop_duplicates(inplace=True)

# remove rows with empty addresses
all_calls_df.dropna(inplace=True)

# combine date and time column
all_calls_df['date_time'] = pd.to_datetime(all_calls_df['date'] + ' ' + all_calls_df['time'], format='%d-%m-%Y %H:%M:%S')
all_calls_df['date'] = pd.to_datetime(all_calls_df['date'], format='%d-%m-%Y')
all_calls_df['time'] = pd.to_datetime(all_calls_df['time'], format='%H:%M:%S')
all_calls_df.set_index('date_time', inplace=True)

# add urgency column
all_calls_df['urgency'] = all_calls_df['descr'].apply(lambda x: str(x).split(':')[0])

# only A1,A2,B1,B2,B calls
urgency_types  = [' Ambulance met hoge spoed',' Ambulance met spoed',
                  ' Ambulance besteld vervoer B1',' Ambulance besteld vervoer B2', 
                  ' Ambulance besteld vervoer']
select_calls_df = all_calls_df[all_calls_df['urgency'].isin(urgency_types)]

select_calls_df.to_csv('20170616_ambulancecalls_2013-2017_cleaned.csv', index=True)
select_calls_df.to_pickle('20170616_ambulancecalls_2013-2017_cleaned.p')

select_calls_df.head()

In [ ]:
del all_calls_df

In [ ]:
fig = plt.figure(figsize=(5,5))

for urge in urgency_types:
    df = select_calls_df[select_calls_df['urgency']==urge]['urgency']
    ax = fig.add_subplot(1, 1, 1)
    ax.set_ylim(0,9000)
    df.resample('M').count().plot(ax=ax, label=urge)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Number of calls per month')
ax.set_title('Number of ambulance calls')

plt.tight_layout()
#plt.savefig('NumberOfCalls_Months.png', dpi=300)

In [ ]:
fig = plt.figure(figsize=(5,5))

for urge in urgency_types:
    df = select_calls_df[select_calls_df['urgency']==urge]['urgency']
    ax = fig.add_subplot(1, 1, 1)
    ax.set_ylim(0,2250)
    df.resample('W').count().plot(ax=ax, label=urge)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Number of calls per week')
ax.set_title('Number of ambulance calls')

plt.tight_layout()
plt.savefig('NumberOfCalls_Weeks.png', dpi=300)

Ambulance calls with high urgency have stayed stable over the last 3 years. 

In [ ]:
fig = plt.figure(figsize=(12,5))

for urge in urgency_types:
    df = select_calls_df[select_calls_df['urgency']==urge]['urgency']
    ax = fig.add_subplot(1, 1, 1)
    ax.set_ylim(0,500)
    df.resample('D').count().plot(ax=ax, label=urge)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Number of calls per day')
ax.set_title('Number of ambulance calls')

plt.tight_layout()
#plt.savefig('NumberOfCalls_Day.png', dpi=300)

### Time series (TS) analysis

https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/  

There's maybe a slight increase in very urgent ambulance calls over time (compare 2013 with 2016)  
Assumptions of TS modeling: mean, variance and covariance should not be a function of time ->   
We can check stationarity using Dickey-Fuller test (null-hypothesis = timeseries NOT stationary):  

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries, windowsize):
    
    #Determing rolling statistics
    #Windowsize = number of observations used
    rolmean = pd.Series.rolling(timeseries, window=windowsize).mean()
    rolstd = pd.Series.rolling(timeseries, window=windowsize).std()

    #Plot rolling statistics:
    plt.figure(figsize=(15,5))
    orig = plt.plot(timeseries, color='steelblue',label='Original')
    mean = plt.plot(rolmean, color='firebrick', label='Rolling Mean')
    std = plt.plot(rolstd, color='dimgrey', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    print('Window size = {} \n'.format(windowsize))
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries)
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    print(dfoutput)

In [ ]:
ts = select_calls_df[select_calls_df['urgency']==' Ambulance met hoge spoed']['urgency']
ts_day = ts.resample('D').count()

In [ ]:
ts_2h = ts.resample('2H').count()

In [ ]:
test_stationarity(ts_2h, 12)

In [ ]:
test_stationarity(ts_day, 30)

In [ ]:
test_stationarity(ts_day, 365)

Dickey-Fuller test is significant, therefore the timeseries can be considered stationary over time and can be used for timeseries modeling.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts_2h)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(15,5))

plt.subplot(411)
plt.plot(ts_day, label='Original')
plt.xlim(['Aug-2014','Dec-2014'])
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.xlim(['Aug-2014','Dec-2014'])
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.xlim(['Aug-2014','Dec-2014'])
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.xlim(['Aug-2014','Dec-2014'])
plt.legend(loc='best')
plt.tight_layout()



Seasonal decomposition also doesn't show a clear trend.

In [ ]:
ts_day_decompose = residual
ts_day_decompose.dropna(inplace=True)
test_stationarity(ts_day_decompose, 30)

In [ ]:
# Plot ACF and PACF
import statsmodels.graphics.tsaplots as tsaplt

fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(211)
fig = tsaplt.plot_acf(ts_day, lags=7, ax=ax1)
ax2 = fig.add_subplot(212)
fig = tsaplt.plot_pacf(ts_day, lags=7, ax=ax2)

In [ ]:
# TS Model

from statsmodels.tsa.arima_model import ARIMA

# ARIMA requires float data instead of integer
ts_day = ts_day.astype(float)

model = ARIMA(ts_day, order=(2, 0, 1))  # order from acf and pacf plots (p,d,q), but 2,0,1 has best fit
results_ARIMA = model.fit(disp=-1)  

plt.figure(figsize=(15,4))
plt.plot(ts_day, color='steelblue')
plt.plot(results_ARIMA.fittedvalues, color='firebrick')
plt.title('Original TS: RSS: %.3f'% sum((results_ARIMA.fittedvalues-ts_day)**2))

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(ts_day['2016'], color='steelblue')
plt.plot(results_ARIMA.fittedvalues['2016'], color='firebrick')
plt.title('Residuals of decomposition: RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_day)**2))

### Counts per day of the week

Looking at the number of calls per day shows a peak at kings day (2017 & 2016) and new years eve in 2017. However, the data is missing for new years day of 2016.

In [ ]:
# mean count per day of week
labels=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
date_list = pd.date_range(select_calls_df.ix[len(select_calls_df)-1,'date'], select_calls_df.ix[0,'date'])

counts = select_calls_df['urgency'].groupby(select_calls_df.index.dayofweek).count()/date_list.dayofweek.value_counts()
counts.plot(kind='bar', color='#7bccc4')
plt.xticks(np.arange(0,7,1), labels, rotation='vertical')
plt.xlabel('Day of the week')
plt.ylabel('Average call count')
plt.title('Call frequency per day of the week')

In [ ]:
labels=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig = plt.figure(figsize=(8,10))

count=1
for urge in urgency_types:
    df = select_calls_df[select_calls_df['urgency']==urge]['urgency']
    date_list = pd.date_range(df.index[len(df)-1].date(), df.index[0].date())
    count_days = date_list.dayofweek.value_counts().sort_index()
    count_calls = df.groupby(df.index.dayofweek).count()
    average_count = count_calls/count_days
    ax = fig.add_subplot(3, 2, count)
    average_count.plot(kind='bar', ax=ax, color='#7bccc4', label=urge)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Average number of calls per day')
    ax.set_xlabel('')
    ax.set_title(urge)
    count+=1

plt.tight_layout()
plt.savefig('NumberOfCalls_DayOfWeek.png', dpi=300)

Most ambulance calls take place on Friday

## Regressors

#### add features to df
- kingsday / new years day 
- day of the week 
- day
- month
- hour


In [ ]:
A1_df = select_calls_df[select_calls_df['urgency']==' Ambulance met hoge spoed']
A1_perday_df = A1_df['urgency'].resample('D').count()
A1_perday_df = A1_perday_df.to_frame()

In [ ]:
A1_perday_df['month'] = A1_perday_df.index.month
A1_perday_df['day'] = A1_perday_df.index.day
A1_perday_df['dayofweek'] = A1_perday_df.index.dayofweek

kingsdays = [datetime(2013,4,30), datetime(2014,4,26), datetime(2015,4,27), datetime(2016,4,27), datetime(2017,4,27)]
A1_perday_df['kingsday'] = [1 if x in kingsdays else 0 for x in A1_perday_df.index]

newyearsdays = [datetime(2014,1,1), datetime(2015,1,1), datetime(2016,1,1), datetime(2017,1,1)]
A1_perday_df['newyears'] = [1 if x in newyearsdays else 0 for x in A1_perday_df.index]

A1_perday_df.rename(columns={'urgency':'daily_count'}, inplace=True)

A1_perday_df.head()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
def plot_regressor(regressor, yvalue):
    fig,ax = plt.subplots(nrows=2, ncols=1, figsize=(18,10))
    ax[0].plot(testDF[yvalue], color='k', label='actual')
    ax[0].plot(testDF['prediction'], color='r', label='prediction (R$^2$ = {0:.3f})'.format(regressor.score(testX, testY)))
    ax[0].legend(loc='best')
    
    ax[1].plot(testDF.loc['2017':, yvalue], color='k', label='actual')
    ax[1].plot(testDF.loc['2017':,'prediction'], color='r')
    
    plt.show()

In [ ]:
def plot_regressor1(regressor, yvalue):
    plt.figure(figsize=(18,5))
    plt.plot(testDF[yvalue], color='k', label='actual')
    plt.plot(testDF['prediction'], color='r', label='prediction (R$^2$ = {0:.3f})'.format(regressor.score(testX, testY)))
    plt.legend(loc='best')
    
    plt.show()

In [ ]:
trainDF = A1_perday_df[:'2016']
testDF = A1_perday_df['2017':]

trainX = trainDF.drop('daily_count', axis=1)
trainY = trainDF['daily_count']

testX = testDF.drop('daily_count', axis=1)
testY = testDF['daily_count']

In [ ]:
gbr = GradientBoostingRegressor(random_state=5)
gbr.fit(trainX, trainY)
testDF['prediction'] = gbr.predict(testX)

plot_regressor1(gbr, 'daily_count')

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

et = ExtraTreesRegressor()
et.fit(trainX, trainY)
testDF['prediction'] = et.predict(testX)

plot_regressor1(et, 'daily_count')

### To do:
- gridsearch
- regressor on residual of TS
- pipeline

In [ ]:
calls2016 = select_calls_df['2016']
test = calls2016['address'].iloc[1:20]
test 

In [ ]:
# use a session to make sure geocoder stays connected and does not give empty results

with requests.Session() as session:
    latlng_test = test.map(lambda x: geocoder.google(x, session=session).latlng)
latlng_test

In [ ]:
latitudes = latlng_test.map(lambda x: x[0])
longitudes = latlng_test.map(lambda x: x[1])

In [ ]:
gmap = gmplot.GoogleMapPlotter(52.3469157,4.8639372,11)
gmap.scatter(latitudes, longitudes, 'black', size=50, marker=False)
gmap.draw("test_calls_map.html")

In [ ]:
from IPython.display import IFrame
IFrame('test_calls_map.html', width=600, height=600)

In [ ]:
# ambulance stands

stands_df = pd.read_csv('Address_ambulancestands.csv')
stands_df['latlng'] = stands_df['address1'].map(lambda x: geocoder.google(x + ', Amsterdam').latlng)
stands_df

In [ ]:
latitudes = stands_df['latlng'].map(lambda x: x[0])
longitudes = stands_df['latlng'].map(lambda x: x[1])

In [ ]:
gmap = gmplot.GoogleMapPlotter(52.3469157,4.8639372,11)
gmap.scatter(latitudes, longitudes, 'black', size=300, marker=False)
gmap.draw("Amsterdam_stands_map.html")

In [ ]:
from IPython.display import IFrame
IFrame('Amsterdam_stands_map.html', width=600, height=600)

In [ ]:
import pickle
pickle.dump(select_calls_df, open('select_calls_df.p','wb'))
select_calls_df = pickle.load(open('select_calls_df.p', 'rb'))

In [ ]:
# TODO: calculate SEM

for index, count in count_calls_perday.iteritems():
    day = index.dayofweek
    sqrd_diff = (count - average_count[day])**2

sum((x - mean)**2)/(N-1)
np.sqrt()